In [1]:
from ten_con import *
from pauli_utils import *

In [2]:
from copy import copy
import numpy as np
from numba import jit

In [41]:
@jit
def mat_decompose(H:np.matrix):
        """Tensorized decomposition of hermit matrix into pauli terms.
            See Hantzko et al, 2023.

        Args:
            H (np.matrix): Hermit matrix.

        Returns:
            np.matrix: Coefficient matrix of the given matrix.
        """

        n1, n2 = H.shape
        assert n1 == n2, "The given matrix must be a square matrix."
        n= int(np.log2(n1))
        l = n1
        for i in range(n):
            m = int(2**i) # Number of submatrix
            l = int(l/2) # Sub matrix size, square
            for j in range(m):
                for k in range(m):
                    num_i = j*(2*l) # Initial position of sub matrix row
                    num_j = k*(2*l) # Initial position of sub matrix column
                    
                    row1 = slice(num_i, num_i +l)
                    row2 = slice(num_i+l, num_i +2*l)
                    col1 = slice(num_j, num_j +l)
                    col2 = slice(num_j+l, num_j +2*l)
                    # I-Z
                    H[row1, col1] += H[row2, col2] 
                    H[row2, col2]  = H[row1, col1] - 2*H[row2, col2]
                    # X-Y
                    H[row1, col2] += H[row2, col1] 
                    H[row2, col1]  = H[row1, col2] - 2*H[row2, col1]
                    H[row2, col1]  *= 1j

        H *= (1/(2**n))
        return H

In [59]:
H = pstr2mat("XZ") + 2j*pstr2mat("XZ") + 0.5*pstr2mat("YY") + pstr2mat("ZI")

In [60]:
H

matrix([[ 1. +0.j,  0. +0.j,  1. +2.j, -0.5+0.j],
        [ 0. +0.j,  1. +0.j,  0.5+0.j, -1. -2.j],
        [ 1. +2.j,  0.5+0.j, -1. +0.j,  0. +0.j],
        [-0.5+0.j, -1. -2.j,  0. +0.j, -1. +0.j]])

In [61]:
%%timeit
mat = copy(H)
mat_decompose_xz(mat)

475 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [62]:
%%timeit
coef = copy(H)
mat_decompose(coef)

8.23 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [63]:
mat = copy(H)
mat_decompose_xz(mat)

matrix([[0. +0.j, 0. +0.j, 1. +0.j, 0. +0.j],
        [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
        [0. +0.j, 1. +2.j, 0. +0.j, 0. +0.j],
        [0. +0.j, 0. +0.j, 0. +0.j, 0.5+0.j]])

In [64]:
coef = copy(H)
mat_decompose(coef)

matrix([[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
        [0. +0.j, 0. +0.j, 0. +0.j, 1. +2.j],
        [0. +0.j, 0. +0.j, 1. +0.j, 0. +0.j],
        [0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j]])

In [65]:
restore = copy(coef)
mat_compose(restore)

matrix([[ 1. +0.j,  0. +0.j,  1. +2.j, -0.5+0.j],
        [ 0. +0.j,  1. +0.j,  0.5+0.j, -1. -2.j],
        [ 1. +2.j,  0.5+0.j, -1. +0.j,  0. +0.j],
        [-0.5+0.j, -1. -2.j,  0. +0.j, -1. +0.j]])

In [51]:
H

matrix([[ 0. +2.j,  0. +0.j,  2. +0.j, -0.5+0.j],
        [ 0. +0.j,  0. -2.j,  0.5+0.j,  0. +0.j],
        [ 2. +0.j,  0.5+0.j,  0. +2.j,  0. +0.j],
        [-0.5+0.j,  0. +0.j,  0. +0.j,  0. -2.j]])

In [58]:
restore = copy(coef)
mat_compose_no_jit(restore)

matrix([[ 0. +0.j,  0. +0.j,  1. +0.j, -0.5+0.j,  0. +0.j,  0. +0.j,
          0. +0.j,  0. +0.j,  1. +2.j,  0. +0.j,  0. +0.j,  0. +0.j,
          0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
        [ 0. +0.j,  0. +0.j,  0.5+0.j,  1. +0.j,  0. +0.j,  0. +0.j,
          0. +0.j,  0. +0.j,  0. +0.j,  1. -2.j,  0. +0.j,  0. +0.j,
          0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
        [ 1. +0.j,  0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
          0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  1. +2.j,  0. +0.j,
          0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
        [-0.5+0.j,  1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
          0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  1. -2.j,
          0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
        [ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         -1. +0.j, -0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         -1. -2.j,  0. +0.j,  0. +0.j,  0. +0.j],
        [ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  

## Numpy

In [2]:
import numpy as np

In [3]:
arr = np.array(
    [
    [0., 1., 0.2, 3.2, 0.1, 0.00001],
    [0., 1., 0.2, 3.2, 0.1, 0.00001],
    [0., 1., 0.2, 3.2, 0.1, 0.00001],
    [0., 1., 0.2, 3.2, 0.1, 0.00001]
    ]
)

In [7]:
np.nonzero(arr>0.1)

(array([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], dtype=int64),
 array([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3], dtype=int64))